# Setup 

In [1]:
import contracts
contracts.disable_all()

import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html

import numpy as np
import geometry as geo



dw.logger.setLevel(50)

INFO:dt-world:duckietown-world 1.0.15
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class GenericData
DEBUG:dt-serialization:Registering class Sequence
DEBUG:dt-serialization:Registering class SampledSequence
DEBUG:dt-serialization:Registering class Constant
DEBUG:dt-serialization:Registering class RectangularArea
DEBUG:dt-serialization:Registering class TransformSequence
DEBUG:dt-serialization:Registering class VariableTransformSequence
DEBUG:dt-serialization:Registering class SE2Transform
DEBUG:dt-serialization:Registering class Scale2D
DEBUG:dt-serialization:Registering class Matrix2D
DEBUG:dt-serialization:Registering class SpatialRelation
DEBUG:dt-serialization:Registering class GroundTruth
DEBUG:dt-serialization:Registering class PlacedObject
DEBUG:dt-serialization:Registering class EvaluatedMetric
DEBUG:dt-serialization:Registering class GenericObject
DEBUG:dt-serialization:Registeri

In [2]:
%%html
<style>
pre {line-height: 90%}
</style>

# Lane segments in field of view

Need:
    
    1. Function that creates field of view polygon (triangle or trapezoid) defined by corner points
    
    2. Function that checks if point is in polygon
    
    3. Function that checks if edge (lane segment) is in polygon by checking for both control points (nodes)


### 1

Function that creates field of view polygon (triangle or trapezoid) defined by corner points

In [3]:
class FOV(dw.PlacedObject):
    import numpy as np
    
    def __init__(self, angle = np.deg2rad(100), height = 1.0, duckie_length = 0.15, duckie_width = 0.15, *args, **kwargs):
        self.angle = angle
        self.height = height
        self.points =  [(duckie_length/2.0, -duckie_width/2.0),
                        ( (duckie_length/2.0  + self.height),
                         -(duckie_width/2.0 + self.height*np.tan(self.angle/2.0)) ),
                        ( (duckie_length/2.0  + self.height),
                          (duckie_width/2.0 + self.height*np.tan(self.angle/2.0)) ),
                        (duckie_length/2.0, duckie_width/2.0)]   

        dw.PlacedObject.__init__(self, *args, **kwargs)
        
    def draw_svg(self, drawing, g):
        c = drawing.polygon(self.points, fill='yellow')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary 
        return self.points



DEBUG:dt-serialization:Registering class FOV


#### Example

In [4]:
def interpolate(q0, q1, alpha):
    v = geo.SE2.algebra_from_group(geo.SE2.multiply(geo.SE2.inverse(q0), q1))
    vi = v * alpha
    q = np.dot(q0, geo.SE2.group_from_algebra(vi))
    return q

In [5]:
q0 = geo.SE2_from_translation_angle([0, 0], 0)
q1 = geo.SE2_from_translation_angle([2, -2], np.deg2rad(-90))

# create a sequence of poses 
n = 10
seqs = []
steps = np.linspace(0, 1, num=n)
for alpha in steps:
    q = interpolate(q0, q1, alpha)
    seqs.append(q)

In [6]:
root = dw.PlacedObject()

In [7]:
timestamps = range(len(seqs)) # [0, 1, 2, ...]

# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs]
seq_fov = dw.SampledSequence(timestamps, transforms)# class used to represent time varying qtties 

In [8]:
root.set_object("FOV", FOV(), ground_truth=seq_fov)
root.set_object('db18-4', dw.DB18(), ground_truth=seq_fov)

In [9]:
area = dw.RectangularArea((-1, -3), (3, 1))

ipython_draw_html(root, area=area);

In [10]:
m = dw.load_map('robotarium1')

sk = dw.get_skeleton_graph(m)

map_root = sk.root2

def draw_graph(G0, pos=None):
    import networkx as nx
    from matplotlib import pyplot as plt
    pos = pos or nx.spring_layout(G0)
    plt.figure(figsize=(12, 12))    
    nx.draw(G0,pos,labels={node:node for node in G0.nodes()})
    def edge_label(a, b):
        datas = G0.get_edge_data(a, b)
        s = '%d edge%s' % (len(datas), 's' if len(datas)>=2 else '')
        for k, v in datas.items():
            if v:
                if 'label' in v:
                    s += '\n %s' % v['label']
                else:
                    s += '\n %s' %v
        return s
    edge_labels = dict([ ((a,b), edge_label(a,b)) for a,b in G0.edges()])
    nx.draw_networkx_edge_labels(G0,pos,edge_labels=edge_labels,font_color='red')
    plt.axis('off')
    plt.show()
    
    

In [11]:
import geometry as geo
pos = {}
angles = {}
for n in sk.G:
    q = sk.G.nodes[n]['point'].as_SE2()
    t, theta = geo.translation_angle_from_SE2(q)
    pos[n] = t
    angles[n] = theta
draw_graph(sk.G, pos=pos)

<Figure size 1200x1200 with 1 Axes>

In [12]:
import networkx as nx

start = 'P60'
end = 'P39'

path = nx.shortest_path(sk.G, start, end)

n = 10
seqs = []

for i in range(len(path)-1):
    q0 = sk.G.nodes[path[i]]['point'].as_SE2()
    q1 = sk.G.nodes[path[i+1]]['point'].as_SE2()
    steps = np.linspace(0, 1, num=n)
    for alpha in steps:
        q = interpolate(q0, q1, alpha)
        seqs.append(q)

In [13]:
timestamps = range(len(seqs)) # [0, 1, 2, ...]

# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs]
seq_fov = dw.SampledSequence(timestamps, transforms)# class used to represent time varying qtties 

In [14]:
map_root.set_object("FOV", FOV(), ground_truth=seq_fov)
map_root.set_object('db18-4', dw.DB18(), ground_truth=seq_fov)
ipython_draw_html(map_root)

### 2

Function that expresses FOV-polygon points in global frame

In [15]:
def FOV2Global(FOV, q):
    global_points = []
    for point in FOV.points:
        q_point = geo.SE2_from_translation_angle(point, 0.0)
        global_point, _ = geo.translation_angle_from_SE2(geo.SE2.multiply(q,q_point))
        global_points.append(global_point)
        
    return global_points        

In [16]:
poly_boundary = FOV2Global(FOV(), sk.G.nodes['P60']['point'].as_SE2())
print(poly_boundary)

[array([1.8437999 , 0.50999998]), array([ 0.65204631, -0.49000002]), array([ 3.18555349, -0.49000002]), array([1.9937999 , 0.50999998])]


Function that checks if point is inside polygon (based on tracing a horizontal line/ray through the point and checking for the intersection) 

In [17]:
def insidePoly(point,poly):
    
    x = point[0]
    y = point[1]
    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

In [18]:
insidePoly (pos['P41'], poly_boundary)

False

### 3

Iterate over all edges and check which are visible

In [19]:
def inFOV(G, poly):
    
    for node1, node2 in G.edges():
        q1 = G.nodes[node1]['point'].as_SE2()
        t1, _ = geo.translation_angle_from_SE2(q1)
        q2 = G.nodes[node1]['point'].as_SE2()
        t2, _ = geo.translation_angle_from_SE2(q2)
        inFOV1 = insidePoly(t1, poly)
        inFOV2 = insidePoly(t2, poly)
        
        if inFOV1 and inFOV2:
            G[node1][node2][0]['inFOV'] = True 
        else:
            G[node1][node2][0]['inFOV'] = False

In [20]:
inFOV(sk.G, poly_boundary)

In [21]:
for node1, node2 in sk.G.edges():
    edge_data = sk.G.get_edge_data(node1, node2).items()
    print(edge_data)

[(0, {'lane': 'ls229', 'inFOV': False})]
[(0, {'lane': 'ls111', 'inFOV': False})]
[(0, {'lane': 'L111', 'inFOV': False})]
[(0, {'lane': 'ls079', 'inFOV': False})]
[(0, {'lane': 'ls234', 'inFOV': False})]
[(0, {'lane': 'ls009', 'inFOV': False})]
[(0, {'lane': 'ls122', 'inFOV': False})]
[(0, {'lane': 'ls245', 'inFOV': False})]
[(0, {'lane': 'ls270', 'inFOV': False})]
[(0, {'lane': 'ls157', 'inFOV': False})]
[(0, {'lane': 'ls134', 'inFOV': False})]
[(0, {'lane': 'ls096', 'inFOV': False})]
[(0, {'lane': 'ls050', 'inFOV': False})]
[(0, {'lane': 'ls260', 'inFOV': False})]
[(0, {'lane': 'ls064', 'inFOV': False})]
[(0, {'lane': 'ls103', 'inFOV': False})]
[(0, {'lane': 'ls062', 'inFOV': False})]
[(0, {'lane': 'ls211', 'inFOV': False})]
[(0, {'lane': 'ls204', 'inFOV': False})]
[(0, {'lane': 'ls127', 'inFOV': False})]
[(0, {'lane': 'L112', 'inFOV': False})]
[(0, {'lane': 'ls135', 'inFOV': False})]
[(0, {'lane': 'ls092', 'inFOV': False})]
[(0, {'lane': 'ls106', 'inFOV': False})]
[(0, {'lane': 'ls1

### 4

New edge attributes: true probability and belief probability

In [29]:
import random 

for node1, node2 in sk.G.edges():
    sk.G[node1][node2][0]['P_true'] = random.randint(0,1)
    edge_data = sk.G.get_edge_data(node1, node2).items()
    print(edge_data)


[(0, {'P_true': 1, 'lane': 'ls229', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls111', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'L111', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls079', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls234', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls009', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'ls122', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'ls245', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls270', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls157', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls134', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'ls096', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'ls050', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls260', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls064', 'inFOV': False})]
[(0, {'P_true': 1, 'lane': 'ls103', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'ls062', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'ls211', 'inFOV': False})]
[(0, {'P_true': 0, 'lane': 'l

Initialize P_berlief

In [30]:
for node1, node2 in sk.G.edges():
    sk.G[node1][node2][0]['P_belief'] = 0.5
    edge_data = sk.G.get_edge_data(node1, node2).items()
    print(edge_data)

[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls229', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls111', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'L111', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls079', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls234', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls009', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls122', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls245', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls270', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls157', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls134', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls096', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls050', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls260', 'inFOV': False})]
[(0, {'

Function that updates belief for segments we can observe

In [31]:
def observation_update(G):
    
    for node1, node2 in G.edges():

        if G[node1][node2][0]['inFOV'] == True:
            G[node1][node2][0]['P_belief']=G[node1][node2][0]['P_true']
            


In [32]:
observation_update(sk.G)

In [33]:
for node1, node2 in sk.G.edges():
    edge_data = sk.G.get_edge_data(node1, node2).items()
    print(edge_data)

[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls229', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls111', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'L111', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls079', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls234', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls009', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls122', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls245', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls270', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls157', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls134', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls096', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 0, 'lane': 'ls050', 'inFOV': False})]
[(0, {'P_belief': 0.5, 'P_true': 1, 'lane': 'ls260', 'inFOV': False})]
[(0, {'

### 5

Update for unobserved segments

### 6

True belief based on imprint of obstacles(duckies/duckiebots...)